### 1. Finding customers who have placed orders with a total value greater than the average order value

__Functional Specification:__
* Main query: Join Sales.Customers (c) → Sales.Orders (o) → Sales.OrderLines (ol)
* Compute total order value per customer as SUM(ol.Quantity * ol.UnitPrice)
* Subquery in HAVING: Calculate average order value across all orders
* Subquery logic: Sum line items per OrderID, then calculate AVG of those totals
* Group by CustomerID and CustomerName to aggregate all orders per customer
* Filter customers whose total order value exceeds the average order value
* Use DISTINCT (though not needed with Sort by TotalOrderValue in descending order
* Result shows customers who spend more than the average order amount


In [ ]:
USE WideWorldImporters;
SELECT DISTINCT 
    c.CustomerID,
    c.CustomerName,
    SUM(ol.Quantity * ol.UnitPrice) AS TotalOrderValue
FROM Sales.Customers c
INNER JOIN Sales.Orders o ON c.CustomerID = o.CustomerID
INNER JOIN Sales.OrderLines ol ON o.OrderID = ol.OrderID
GROUP BY c.CustomerID, c.CustomerName
HAVING SUM(ol.Quantity * ol.UnitPrice) > (
    SELECT AVG(OrderTotal)
    FROM (
        SELECT SUM(Quantity * UnitPrice) AS OrderTotal
        FROM Sales.OrderLines
        GROUP BY OrderID
    ) AS AvgOrders
)
ORDER BY TotalOrderValue DESC;


### 2. Finding suppliers who supply products that are currently almost running out of stock, with less than 100 items

__Functional Specification:__
* Main query: SELECT from Purchasing.Suppliers (s)
* Subquery: Find SupplierIDs that have products with low stock
* Subquery: joins Warehouse.StockItems (si) → Warehouse.StockItemHoldings (sih)
* Filter products where QuantityOnHand < 100
* Main query filters suppliers using IN clause with subquery results
* Use DISTINCT to avoid duplicate suppliers if they have multiple low-stock products
* Sort by SupplierName alphabetically
* Result shows supplier details (ID, name, phone) for suppliers with low-stock items

In [ ]:
USE WideWorldImporters;
SELECT DISTINCT
    s.SupplierID,
    s.SupplierName,
    s.PhoneNumber
FROM Purchasing.Suppliers s
WHERE s.SupplierID IN (
    SELECT si.SupplierID
    FROM Warehouse.StockItems si
    INNER JOIN Warehouse.StockItemHoldings sih ON si.StockItemID = sih.StockItemID
    WHERE sih.QuantityOnHand < 100
)
ORDER BY s.SupplierName;

### 3. List employees who have processed more orders than the average employee

__Functional Specification:__
* Main query: Join Application.People (p) → Sales.Orders (o) on PickedByPersonID
* Count orders processed per employee using COUNT(o.OrderID)
* Subquery 1 (in SELECT): Calculate average orders per employee across all employees
* Subquery 2 (in HAVING): Same calculation to filter employees above average
* Subquery logic: Count orders per employee, then calculate AVG of those counts
* Group by PersonID and FullName to aggregate orders per employee
* Filter using HAVING to show only employees above average
* Sort by OrdersProcessed in descending order
* Result shows employee details, their order count, and the average for comparison


In [ ]:
USE WideWorldImporters;
SELECT 
    p.PersonID,
    p.FullName,
    COUNT(o.OrderID) AS OrdersProcessed,
    (
        SELECT AVG(OrderCount)
        FROM (
            SELECT COUNT(*) AS OrderCount
            FROM Sales.Orders
            GROUP BY PickedByPersonID
        ) AS AvgOrders
    ) AS AvgOrdersPerEmployee
FROM Application.People p
INNER JOIN Sales.Orders o ON p.PersonID = o.PickedByPersonID
GROUP BY p.PersonID, p.FullName
HAVING COUNT(o.OrderID) > (
    SELECT AVG(OrderCount)
    FROM (
        SELECT COUNT(*) AS OrderCount
        FROM Sales.Orders
        GROUP BY PickedByPersonID
    ) AS AvgOrders
)
ORDER BY OrdersProcessed DESC;

### 4: Yearly Sales Performance Summary

__Functional Specification:__
* CTE: YearlySales aggregates sales metrics by year
* Join Sales.Orders (o) → Sales.OrderLines (ol)
* Extract year using YEAR(o.OrderDate)
* Calculate yearly total as SUM(ol.Quantity * ol.UnitPrice)
* Count distinct orders using COUNT(DISTINCT o.OrderID)
* Group by OrderYear
* Main query computes AvgOrderValue = YearlyTotal / TotalOrders
* Round YearlyTotal and AvgOrderValue to 2 decimal places
* Sort by OrderYear chronologically

In [ ]:

USE WideWorldImporters;
WITH YearlySales AS (
    SELECT 
        YEAR(o.OrderDate) AS OrderYear,
        SUM(ol.Quantity * ol.UnitPrice) AS YearlyTotal,
        COUNT(DISTINCT o.OrderID) AS TotalOrders
    FROM Sales.Orders o
    INNER JOIN Sales.OrderLines ol ON o.OrderID = ol.OrderID
    GROUP BY YEAR(o.OrderDate)
)
SELECT 
    OrderYear,
    ROUND(YearlyTotal, 2) AS YearlyTotal,
    TotalOrders,
    ROUND(YearlyTotal / TotalOrders, 2) AS AvgOrderValue
FROM YearlySales
ORDER BY OrderYear;


### 5: Products with Total Quantity Sold

__Functional Specification:__
* CTE: ProductSales aggregates quantity sold per product
* Join Warehouse.StockItems (si) → Sales.OrderLines (ol)
* Calculate total quantity as SUM(ol.Quantity)
* Group by StockItemID and StockItemName
* Main query filters products with TotalQuantitySold > 100
* Sort by TotalQuantitySold in descending order

In [ ]:

USE WideWorldImporters;
WITH ProductSales AS (
    SELECT 
        si.StockItemID,
        si.StockItemName,
        SUM(ol.Quantity) AS TotalQuantitySold
    FROM Warehouse.StockItems si
    INNER JOIN Sales.OrderLines ol ON si.StockItemID = ol.StockItemID
    GROUP BY si.StockItemID, si.StockItemName
)
SELECT 
    StockItemName,
    TotalQuantitySold
FROM ProductSales
WHERE TotalQuantitySold > 100
ORDER BY TotalQuantitySold DESC;

### 6: Monthly Order Count Summary

__Functional Specification:__
* CTE: MonthlyOrders counts orders per month
* Extract year using YEAR(OrderDate) and month using MONTH(OrderDate)
* Count total orders using COUNT(*)
* Group by OrderYear and OrderMonth
* Main query returns all monthly order counts
* Sort by OrderYear and OrderMonth chronologically

In [ ]:

USE WideWorldImporters;
WITH MonthlyOrders AS (
    SELECT 
        YEAR(OrderDate) AS OrderYear,
        MONTH(OrderDate) AS OrderMonth,
        COUNT(*) AS OrderCount
    FROM Sales.Orders
    GROUP BY YEAR(OrderDate), MONTH(OrderDate)
)
SELECT 
    OrderYear,
    OrderMonth,
    OrderCount
FROM MonthlyOrders
ORDER BY OrderYear, OrderMonth;


### 7: High-Volume Sales Employees

__Functional Specification:__
* CTE: EmployeeOrders counts orders handled by each employee
* Join Application.People (p) → Sales.Orders (o) on SalespersonPersonID
* Filter only employees using WHERE p.IsEmployee = 1
* Count orders using COUNT(o.OrderID)
* Group by PersonID and FullName
* Main query filters employees with TotalOrders > 50
* Sort by TotalOrders in descending order

In [ ]:
USE WideWorldImporters;
WITH EmployeeOrders AS (
    SELECT 
        p.PersonID,
        p.FullName,
        COUNT(o.OrderID) AS TotalOrders
    FROM Application.People p
    INNER JOIN Sales.Orders o ON p.PersonID = o.SalespersonPersonID
    WHERE p.IsEmployee = 1
    GROUP BY p.PersonID, p.FullName
)
SELECT 
    FullName,
    TotalOrders
FROM EmployeeOrders
WHERE TotalOrders > 50
ORDER BY TotalOrders DESC;

### 8: Average Order Value by Customer

__Functional Specification:__
* CTE: CustomerOrderValues calculates order metrics per customer
* Join Sales.Customers (c) → Sales.Orders (o) → Sales.OrderLines (ol)
* Count distinct orders using COUNT(DISTINCT o.OrderID)
* Calculate total spending as SUM(ol.Quantity * ol.UnitPrice)
* Group by CustomerID and CustomerName
* Main query computes AvgOrderValue = TotalSpent / OrderCount
* Round AvgOrderValue to 2 decimal places
* Sort by AvgOrderValue in descending order

In [ ]:
USE WideWorldImporters;
WITH CustomerOrderValues AS (
    SELECT 
        c.CustomerID,
        c.CustomerName,
        COUNT(DISTINCT o.OrderID) AS OrderCount,
        SUM(ol.Quantity * ol.UnitPrice) AS TotalSpent
    FROM Sales.Customers c
    INNER JOIN Sales.Orders o ON c.CustomerID = o.CustomerID
    INNER JOIN Sales.OrderLines ol ON o.OrderID = ol.OrderID
    GROUP BY c.CustomerID, c.CustomerName
)
SELECT 
    CustomerName,
    OrderCount,
    TotalSpent,
    ROUND(TotalSpent / OrderCount, 2) AS AvgOrderValue
FROM CustomerOrderValues
ORDER BY AvgOrderValue DESC;


### 9: Top 5 Products by Revenue

__Functional Specification:__
* CTE: ProductRevenue calculates total revenue per product
* Join Warehouse.StockItems (si) → Sales.OrderLines (ol)
* Calculate revenue as SUM(ol.Quantity * ol.UnitPrice)
* Group by StockItemName
* Main query selects TOP 5 products by revenue
* Round TotalRevenue to 2 decimal places
* Sort by TotalRevenue in descending order

In [ ]:
USE WideWorldImporters;
WITH ProductRevenue AS (
    SELECT 
        si.StockItemName,
        SUM(ol.Quantity * ol.UnitPrice) AS TotalRevenue
    FROM Warehouse.StockItems si
    INNER JOIN Sales.OrderLines ol ON si.StockItemID = ol.StockItemID
    GROUP BY si.StockItemName
)
SELECT TOP 5
    StockItemName,
    ROUND(TotalRevenue, 2) AS TotalRevenue
FROM ProductRevenue
ORDER BY TotalRevenue DESC;

### 10: Inactive Customers (No Orders in 6 Months)

Functional Specification:
* CTE: LastOrderDate finds most recent order per customer
* Join Sales.Customers (c) → Sales.Orders (o)
* Find last order date using MAX(o.OrderDate)
* Calculate days since last order using DATEDIFF(DAY, MAX(o.OrderDate), GETDATE())
* Group by CustomerID and CustomerName
* Main query filters customers with DaysSinceLastOrder > 180
* Sort by DaysSinceLastOrder in descending order

In [ ]:
USE WideWorldImporters;
WITH LastOrderDate AS (
    SELECT 
        c.CustomerID,
        c.CustomerName,
        MAX(o.OrderDate) AS LastOrder,
        DATEDIFF(DAY, MAX(o.OrderDate), GETDATE()) AS DaysSinceLastOrder
    FROM Sales.Customers c
    INNER JOIN Sales.Orders o ON c.CustomerID = o.CustomerID
    GROUP BY c.CustomerID, c.CustomerName
)
SELECT 
    CustomerName,
    LastOrder,
    DaysSinceLastOrder
FROM LastOrderDate
WHERE DaysSinceLastOrder > 180
ORDER BY DaysSinceLastOrder DESC;